In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, LSTM, GRU
from sklearn.utils import resample
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
# Load the dataset
df = pd.read_csv('cleveland297.csv', header = None)

df.columns = ['age', 'sex', 'cp', 'trestbps', 'chol',
              'fbs', 'restecg', 'thalach', 'exang', 
              'oldpeak', 'slope', 'ca', 'thal', 'target']

df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,6,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,3,2
2,67,1,4,120,229,0,2,129,1,2.6,2,2,7,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0,3,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,57,0,4,140,241,0,0,123,1,0.2,2,0,7,1
293,45,1,1,110,264,0,0,132,0,1.2,2,0,7,1
294,68,1,4,144,193,1,0,141,0,3.4,2,2,7,2
295,57,1,4,130,131,0,0,115,1,1.2,2,1,7,3


In [3]:
### 1 = male, 0 = female
df.isnull().sum()

df['target'] = df.target.map({0: 0, 1: 1, 2: 1, 3: 1, 4: 1})
df['sex'] = df.sex.map({0: 'female', 1: 'male'})
df['thal'] = df.thal.fillna(df.thal.mean())
df['ca'] = df.ca.fillna(df.ca.mean())
df['sex'] = df.sex.map({'female': 0, 'male': 1})

In [4]:
# Separate features and target variable
X = df.drop('target', axis=1)
y = df['target']


In [5]:

# 1. Load and preprocess the data
# Assuming you have your data in the X and y variables

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



In [6]:
input_shape = (X_train.shape[1], 1)
input_shape_cnn = (X_train.shape[1], 1)

In [7]:
# 4. Hyperparameter tuning
learning_rate = 0.001
batch_size = 50
epochs = 100



# 5. Learning rate scheduling
lr_scheduler = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate, decay_steps=10000, decay_rate=0.9
)
optimizer = keras.optimizers.Adam(learning_rate=lr_scheduler)



# 6. Regularization techniques
callback_early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)
callback_checkpoint = keras.callbacks.ModelCheckpoint(
    'model_checkpoint.h5', save_best_only=True
)



In [8]:
def build_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv1D(filters=11, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.25))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [9]:
# Train the individual models
cnn_model = build_cnn_model(input_shape)
cnn_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),
             callbacks=[callback_early_stopping, callback_checkpoint])


Epoch 1/100
5/5 [==============================] - 2s 134ms/step - loss: 0.7096 - accuracy: 0.6118 - val_loss: 0.6680 - val_accuracy: 0.7667
Epoch 2/100
5/5 [==============================] - 0s 37ms/step - loss: 0.5651 - accuracy: 0.7004 - val_loss: 0.6498 - val_accuracy: 0.8500
Epoch 3/100
5/5 [==============================] - 0s 35ms/step - loss: 0.5307 - accuracy: 0.7722 - val_loss: 0.6384 - val_accuracy: 0.8833
Epoch 4/100
5/5 [==============================] - 0s 35ms/step - loss: 0.4914 - accuracy: 0.7722 - val_loss: 0.6341 - val_accuracy: 0.8500
Epoch 5/100
5/5 [==============================] - 0s 35ms/step - loss: 0.4704 - accuracy: 0.7806 - val_loss: 0.6268 - val_accuracy: 0.8000
Epoch 6/100
5/5 [==============================] - 0s 35ms/step - loss: 0.4543 - accuracy: 0.7764 - val_loss: 0.6207 - val_accuracy: 0.7667
Epoch 7/100
5/5 [==============================] - 0s 40ms/step - loss: 0.5240 - accuracy: 0.7679 - val_loss: 0.6171 - val_accuracy: 0.7667
Epoch 8/100
5/5 [==

In [10]:
# 8. Evaluate the model
test_loss, test_accuracy = cnn_model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


2/2 [==============================] - 0s 0s/step - loss: 0.4407 - accuracy: 0.8333
Test Loss: 0.4407
Test Accuracy: 0.8333


In [11]:
# Make predictions
y_pred = cnn_model.predict(X_test)
y_pred_classes = np.round(y_pred)


# Calculate confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report


conf_matrix = confusion_matrix(y_test, y_pred_classes)
class_report = classification_report(y_test, y_pred_classes)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


2/2 [==============================] - 0s 0s/step
Confusion Matrix:
[[34  2]
 [ 8 16]]

Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.94      0.87        36
           1       0.89      0.67      0.76        24

    accuracy                           0.83        60
   macro avg       0.85      0.81      0.82        60
weighted avg       0.84      0.83      0.83        60



In [12]:
# 4. Hyperparameter tuning
learning_rate = 0.001
batch_size = 50
epochs = 100



# 5. Learning rate scheduling
lr_scheduler = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate, decay_steps=10000, decay_rate=0.9
)
optimizer = keras.optimizers.Adam(learning_rate=lr_scheduler)



# 6. Regularization techniques
callback_early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)
callback_checkpoint = keras.callbacks.ModelCheckpoint(
    'model_checkpoint.h5', save_best_only=True
)



In [13]:
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(11, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(32, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(LSTM(64, return_sequences=True))
    model.add(LSTM(32))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [14]:
lstm_model = build_lstm_model(input_shape)
lstm_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),
              callbacks=[callback_early_stopping, callback_checkpoint])


Epoch 1/100
5/5 [==============================] - 13s 650ms/step - loss: 0.6762 - accuracy: 0.6624 - val_loss: 0.6815 - val_accuracy: 0.6333
Epoch 2/100
5/5 [==============================] - 0s 73ms/step - loss: 0.6277 - accuracy: 0.7215 - val_loss: 0.6567 - val_accuracy: 0.6667
Epoch 3/100
5/5 [==============================] - 0s 73ms/step - loss: 0.5806 - accuracy: 0.7384 - val_loss: 0.6240 - val_accuracy: 0.6667
Epoch 4/100
5/5 [==============================] - 0s 76ms/step - loss: 0.5772 - accuracy: 0.7131 - val_loss: 0.6096 - val_accuracy: 0.7000
Epoch 5/100
5/5 [==============================] - 0s 47ms/step - loss: 0.5675 - accuracy: 0.7342 - val_loss: 0.6184 - val_accuracy: 0.6667
Epoch 6/100
5/5 [==============================] - 0s 45ms/step - loss: 0.5557 - accuracy: 0.7300 - val_loss: 0.6253 - val_accuracy: 0.6000
Epoch 7/100
5/5 [==============================] - 0s 46ms/step - loss: 0.5729 - accuracy: 0.6793 - val_loss: 0.6352 - val_accuracy: 0.6000
Epoch 8/100
5/5 [=

In [15]:
# 8. Evaluate the model
test_loss, test_accuracy = lstm_model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


2/2 [==============================] - 0s 12ms/step - loss: 0.5885 - accuracy: 0.6667
Test Loss: 0.5885
Test Accuracy: 0.6667


In [16]:
# Make predictions
y_pred = lstm_model.predict(X_test)
y_pred_classes = np.round(y_pred)


# Calculate confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report


conf_matrix = confusion_matrix(y_test, y_pred_classes)
class_report = classification_report(y_test, y_pred_classes)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


2/2 [==============================] - 2s 10ms/step
Confusion Matrix:
[[36  0]
 [20  4]]

Classification Report:
              precision    recall  f1-score   support

           0       0.64      1.00      0.78        36
           1       1.00      0.17      0.29        24

    accuracy                           0.67        60
   macro avg       0.82      0.58      0.53        60
weighted avg       0.79      0.67      0.58        60



In [17]:
# 4. Hyperparameter tuning
learning_rate = 0.001
batch_size = 50
epochs = 100



# 5. Learning rate scheduling
lr_scheduler = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate, decay_steps=10000, decay_rate=0.9
)
optimizer = keras.optimizers.Adam(learning_rate=lr_scheduler)



# 6. Regularization techniques
callback_early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)
callback_checkpoint = keras.callbacks.ModelCheckpoint(
    'model_checkpoint.h5', save_best_only=True
)



In [18]:
def build_gru_model(input_shape):
    model = Sequential()
    model.add(GRU(11, return_sequences=True, input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(32, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(64, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(128, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(128))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [19]:
gru_model = build_gru_model(input_shape)
gru_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),
             callbacks=[callback_early_stopping, callback_checkpoint])


Epoch 1/100
5/5 [==============================] - 20s 854ms/step - loss: 0.6439 - accuracy: 0.6034 - val_loss: 0.6828 - val_accuracy: 0.6833
Epoch 2/100
5/5 [==============================] - 1s 123ms/step - loss: 0.5586 - accuracy: 0.7173 - val_loss: 0.6791 - val_accuracy: 0.6833
Epoch 3/100
5/5 [==============================] - 1s 128ms/step - loss: 0.4883 - accuracy: 0.8059 - val_loss: 0.6781 - val_accuracy: 0.6833
Epoch 4/100
5/5 [==============================] - 0s 88ms/step - loss: 0.4999 - accuracy: 0.7722 - val_loss: 0.6782 - val_accuracy: 0.7667
Epoch 5/100
5/5 [==============================] - 1s 126ms/step - loss: 0.4648 - accuracy: 0.7511 - val_loss: 0.6766 - val_accuracy: 0.8000
Epoch 6/100
5/5 [==============================] - 1s 128ms/step - loss: 0.4397 - accuracy: 0.7848 - val_loss: 0.6758 - val_accuracy: 0.8000
Epoch 7/100
5/5 [==============================] - 1s 131ms/step - loss: 0.4344 - accuracy: 0.7806 - val_loss: 0.6755 - val_accuracy: 0.8500
Epoch 8/100
5

Epoch 59/100
5/5 [==============================] - 0s 81ms/step - loss: 0.1762 - accuracy: 0.9409 - val_loss: 0.3653 - val_accuracy: 0.8333
Epoch 60/100
5/5 [==============================] - 0s 82ms/step - loss: 0.2037 - accuracy: 0.9030 - val_loss: 0.3928 - val_accuracy: 0.8333
Epoch 61/100
5/5 [==============================] - 0s 86ms/step - loss: 0.1859 - accuracy: 0.9198 - val_loss: 0.3681 - val_accuracy: 0.8500
Epoch 62/100
5/5 [==============================] - 1s 148ms/step - loss: 0.2045 - accuracy: 0.9241 - val_loss: 0.3175 - val_accuracy: 0.8500
Epoch 63/100
5/5 [==============================] - 1s 133ms/step - loss: 0.1646 - accuracy: 0.9409 - val_loss: 0.3055 - val_accuracy: 0.8833
Epoch 64/100
5/5 [==============================] - 0s 96ms/step - loss: 0.1214 - accuracy: 0.9578 - val_loss: 0.3238 - val_accuracy: 0.8333
Epoch 65/100
5/5 [==============================] - 0s 80ms/step - loss: 0.1792 - accuracy: 0.9409 - val_loss: 0.3529 - val_accuracy: 0.8500
Epoch 66/10

In [20]:
# 8. Evaluate the model
test_loss, test_accuracy = gru_model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


2/2 [==============================] - 0s 20ms/step - loss: 0.3055 - accuracy: 0.8833
Test Loss: 0.3055
Test Accuracy: 0.8833


In [21]:
# Make predictions
y_pred = gru_model.predict(X_test)
y_pred_classes = np.round(y_pred)


# Calculate confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report


conf_matrix = confusion_matrix(y_test, y_pred_classes)
class_report = classification_report(y_test, y_pred_classes)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


2/2 [==============================] - 3s 12ms/step
Confusion Matrix:
[[35  1]
 [ 6 18]]

Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.97      0.91        36
           1       0.95      0.75      0.84        24

    accuracy                           0.88        60
   macro avg       0.90      0.86      0.87        60
weighted avg       0.89      0.88      0.88        60



In [22]:
# Make predictions using individual models
cnn_predictions = (cnn_model.predict(X_test) > 0.5).astype("int32")
lstm_predictions = (lstm_model.predict(X_test) > 0.5).astype("int32")
gru_predictions = (gru_model.predict(X_test) > 0.5).astype("int32")



2/2 [==============================] - 0s 10ms/step


In [23]:
# Combine predictions using a majority voting ensemble
ensemble_predictions = np.mean([cnn_predictions, lstm_predictions, gru_predictions], axis=0) >= 0.5
ensemble_predictions = ensemble_predictions.astype("int32")

In [24]:
# Calculate accuracy of the ensemble
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Ensemble Accuracy: 0.8167


In [25]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, ensemble_predictions)

print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
class_report = classification_report(y_test, ensemble_predictions)
print("Classification Report:")
print(class_report)

# Evaluate ensemble accuracy
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")


Confusion Matrix:
[[35  1]
 [10 14]]
Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.97      0.86        36
           1       0.93      0.58      0.72        24

    accuracy                           0.82        60
   macro avg       0.86      0.78      0.79        60
weighted avg       0.84      0.82      0.81        60

Ensemble Accuracy: 0.8167


In [26]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.utils import resample
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from sklearn.model_selection import cross_val_score

In [27]:
# List of base models

lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)

svm_l_model = SVC(kernel="linear")
svm_l_model.fit(X_train, y_train)

svm_p_model = SVC(kernel="poly")
svm_p_model.fit(X_train, y_train)

svm_r_model = SVC(kernel="rbf")
svm_r_model.fit(X_train, y_train)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)

gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)

ab_model = AdaBoostClassifier(n_estimators=100, random_state=42)
ab_model.fit(X_train, y_train)

xgb_model = XGBClassifier(n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)

gpc_model = GaussianProcessClassifier(random_state=42)
gpc_model.fit(X_train, y_train)

GaussianProcessClassifier(random_state=42)

In [28]:
lr_predictions = lr_model.predict(X_test)
svm_l_predictions = svm_l_model.predict(X_test)
svm_p_predictions = svm_p_model.predict(X_test)
svm_r_predictions = svm_r_model.predict(X_test)
rf_predictions = rf_model.predict(X_test)
knn_predictions = knn_model.predict(X_test)
gb_predictions = gb_model.predict(X_test)
ab_predictions = ab_model.predict(X_test)
xgb_predictions = xgb_model.predict(X_test)
gpc_predictions = gpc_model.predict(X_test)


In [29]:
# Combine predictions using ensemble (Voting Classifier)
ensemble_predictions = np.column_stack((cnn_predictions, lstm_predictions, gru_predictions, lr_predictions, 
                                        svm_l_predictions, svm_p_predictions, svm_r_predictions, 
                                        rf_predictions, knn_predictions, gb_predictions, 
                                        ab_predictions, xgb_predictions, gpc_predictions))


In [30]:
# Perform majority voting to get the final prediction
final_predictions = np.mean(ensemble_predictions, axis=1) >= 0.5
final_predictions = final_predictions.astype("int32")

In [31]:
# Calculate accuracy of the ensemble
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Ensemble Accuracy: 0.9000


In [32]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, final_predictions)

print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
class_report = classification_report(y_test, final_predictions)
print("Classification Report:")
print(class_report)

# Evaluate ensemble accuracy
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Confusion Matrix:
[[34  2]
 [ 4 20]]
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.94      0.92        36
           1       0.91      0.83      0.87        24

    accuracy                           0.90        60
   macro avg       0.90      0.89      0.89        60
weighted avg       0.90      0.90      0.90        60

Ensemble Accuracy: 0.9000


In [33]:
# Combine predictions using ensemble (Voting Classifier)
ensemble_predictions = np.column_stack((cnn_predictions, lstm_predictions, gru_predictions, svm_r_predictions, 
                                        rf_predictions, knn_predictions, gb_predictions, 
                                        xgb_predictions, gpc_predictions))


In [34]:
# Perform majority voting to get the final prediction
final_predictions = np.mean(ensemble_predictions, axis=1) >= 0.5
final_predictions = final_predictions.astype("int32")

# Calculate accuracy of the ensemble
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, final_predictions)

print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
class_report = classification_report(y_test, final_predictions)
print("Classification Report:")
print(class_report)

# Evaluate ensemble accuracy
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Ensemble Accuracy: 0.8833
Confusion Matrix:
[[34  2]
 [ 5 19]]
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.94      0.91        36
           1       0.90      0.79      0.84        24

    accuracy                           0.88        60
   macro avg       0.89      0.87      0.88        60
weighted avg       0.88      0.88      0.88        60

Ensemble Accuracy: 0.8833
